## Download data from [here](https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def create_generators(batch_size, train_data_path, val_data_path):

  train_preprocessor = ImageDataGenerator(
      rescale= 1 / 255.,
      rotation_range=10,
      width_shift_range=0.1,
  )

  val_preprocessor = ImageDataGenerator(
      rescale=1/255.
  )

  train_generator = train_preprocessor.flow_from_directory(
      train_data_path,
      class_mode="categorical",
      target_size=(512,512),
      color_mode='rgb',
      shuffle=True,
      batch_size=batch_size
  )

  val_generator = val_preprocessor.flow_from_directory(
        val_data_path,
        class_mode="categorical",
        target_size=(512,512),
        color_mode="rgb",
        shuffle=False,
        batch_size=batch_size,
    )
  
  return train_generator, val_generator

In [ ]:
path_to_train = "/content/drive/MyDrive/Colab Notebooks/data/Brain_tumor_MRI/Training"
path_to_val = "/content/drive/MyDrive/Colab Notebooks/data/Brain_tumor_MRI/Testing"

batch_size = 16

train_generator, val_generator = create_generators(batch_size, path_to_train, path_to_val)
nbr_classes = train_generator.num_classes

print("Number of classes inside the dataset is : ", nbr_classes)

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Number of classes inside the dataset is :  4


In [ ]:
from tensorflow.keras.layers import Conv2D, Input, Dense, MaxPool2D, BatchNormalization, GlobalAvgPool2D
from tensorflow.keras import Model

def get_model(number_classes):

  my_input = Input(shape=(512,512,3))

  x = Conv2D(32, (3,3), activation="relu")(my_input)
  x = MaxPool2D()(x)
  x = BatchNormalization()(x)

  x = Conv2D(64, (3,3), activation="relu")(my_input)
  x = MaxPool2D()(x)
  x = BatchNormalization()(x)

  x = Conv2D(128, (3,3), activation="relu")(my_input)
  x = MaxPool2D()(x)
  x = BatchNormalization()(x)

  x = GlobalAvgPool2D()(x)
  x = Dense(128, activation="relu")(x)
  x = Dense(number_classes, activation="softmax")(x) # probabilities

  return Model(inputs=my_input, outputs=x)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

path_to_save_model = "/content/drive/MyDrive/Colab Notebooks/data/Models"
ckpt_saver = ModelCheckpoint(
    path_to_save_model, 
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_freq="epoch",
    verbose=1
)

early_stop = EarlyStopping(monitor="val_accuracy", patience=10)

In [ ]:
epochs = 20
lr = 0.0001
CONTINUE_FROM_LAST_MODEL = False

if CONTINUE_FROM_LAST_MODEL:
  model = tf.keras.models.load_model(path_to_save_model)
else:
  model = get_model(number_classes=nbr_classes)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr, amsgrad=True)

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])

history = model.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[ckpt_saver, early_stop]
)

## Evaluating the model

In [ ]:
model = tf.keras.models.load_model(path_to_save_model)

# Evaluate on the validation set
loss_value, accuracy = model.evaluate(val_generator)
print("val_loss : ", loss_value)
print("accuracy : {}%".format(accuracy * 100))

# Evaluate on the training set
loss_value, accuracy = model.evaluate(train_generator)
print("val_loss : ", loss_value)
print("accuracy : {}%".format(accuracy * 100))

82/82 [==============================] - 173s 2s/step - loss: 0.5357 - accuracy: 0.8070
val_loss :  0.5357498526573181
accuracy : 80.70175647735596%
357/357 [==============================] - 1859s 5s/step - loss: 0.3259 - accuracy: 0.8922
val_loss :  0.3258569538593292
accuracy : 89.21568393707275%


## Using our model to predict on single images

In [ ]:
image_path = "/content/drive/MyDrive/Colab Notebooks/data/Brain_tumor_MRI/Testing/glioma/Te-gl_0062.jpg"

image = tf.keras.utils.load_img(image_path, target_size=(512,512))
image = tf.cast(tf.keras.utils.img_to_array(image), dtype=tf.float32) / 255.
image = tf.expand_dims(image, axis=0) # (512,512,3) ==> (1,512,512,3)

result = model(image)
print(result[0])
print(train_generator.class_indices)

tf.Tensor([0.9901787  0.00315967 0.00167303 0.00498859], shape=(4,), dtype=float32)
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
